# GDMA Project
Author: Julian Schelb (1069967)

In [1]:
from neo4j import GraphDatabase
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Connection to the database instance

In [2]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "subatomic-shrank-Respond"))
database_name = "cddb"

***

### Task 1: Import Data

*Task description:* In this project you are given the data of FreeDB (cddb) that contains information about music records. The data is stored in a relational database, i.e.,
PostgreSQL. You will find the sql dump file to load the data into your local
PostgresSQl at the course website. After importing the data into Postgres you
need to accomplish the following tasks

Write code or describe the operations in order to load the data into Neo4j. Note
that it is not obligatory to write SQL Code. It is totally acceptable if you decide
to export the database to CSV files and then use the CSV files to import the
data into Neo4j. However, you need to provide a sufficient amount of details on
how you modelled the data in Neo4j and elaborate on all the implementation
and design decisions you took. Note that for this task, you are not allowed to
use any external tools, e.g. the ETL tool of Neo4j.

***

#### Design Decisions

##### Graph Schema:
![](Figures/cddb_as_graph-Default.png)

##### Mappping of Tables
Entries in tables *artists*, *albums*, *songs* and *cds* represent entities and are mapped to nodes of appropriate type. The cross table  *cdtracks* is mapped as relations between CDs and Songs with the property *track*. The cross table  *artist2album* is mapped as a triangular relationships, using six relations each to connect Artists, Albums and CDs in both directions.

##### Bidirectional Relations

To consgtruct the queries necessary for subsequent tasks, it was structly speaking not necessary to create relations for both directions. However, doing so allows for more readable queries. In addition, the relational SQL database does not define a direction for relations either, and this property is thereby preserved.

##### Unique Identifiers
The unique identifiers present in the SQL database are preserved as node properties. Although Neo4j automatically assigns different IDs internally, this preserves compatibility with the SQL version. It is conceivable that in the real worlds, queries may reference those IDs directly. For example as hardcoded list as part of a GUI.

##### Entity Names
As in the SQL version, the actual name of a *song*, *genre*, *album* and *artist* is a property named after the entity type and therefore different per node label. This was done to map the SQL version as closely as possible to the graph version. In real world scenarios it may be beneficial to rename these properties to *name* across all nodes. This would allow for simpler queries in cases where nodes of different types are filtered by name. Also, a joint full-text index would be possible.

##### Indices
To speed up *MERGE* and *MATCH* operations, the IDs for nodes with label *Artist*, *Album*, *Genre*, *Song* or *CD* are indexed. In addition, a full text index is used for the names of *Artists*, *Albums* and *Songs* to allow for imprecise searches (fuzzy search).

##### Importing Songs

Songs with a trailing backslashcoused problems because the closing quote is escaped. To mitigate this, an extra whitespace character is added to those song titles. Effected rows can be found with the following sql statement:

``` sql
SELECT * 
from cddb.songs s 
where song like '%\\' 
  and song not like '%\\\\'
```

Statement to update the rows:

``` sql
UPDATE cddb.songs 
SET song = song || ' ' 
WHERE song LIKE '%\\' 
  AND song NOT LIKE '%\\\\'
```

#### Remove all Nodes

In [3]:
query = """
DROP DATABASE CDDB IF EXISTS
"""

with driver.session() as session:
    result = session.run(query)

In [4]:
query = """
CREATE DATABASE CDDB IF NOT EXISTS
"""

with driver.session() as session:
    result = session.run(query)

In [5]:
query = """
match (a) delete a
"""

with driver.session(database=database_name) as session:
    result = session.run(query)
    
    
query = """
match (a) -[r] -> () delete a, r
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

#### Import Albums

In [6]:
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///albums_202206062116.csv' AS row FIELDTERMINATOR ','
WITH toInteger(row.albumid) as albumid, row.album as album SKIP 0 LIMIT 100000
CREATE (n:Album {id: albumid, album: album})
//RETURN albumid, album
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

Create fulltext index enabling fuzzy search:  
(see https://neo4j.com/docs/cypher-manual/current/indexes-for-full-text-search/)

In [7]:
query = """
CREATE FULLTEXT INDEX albums IF NOT EXISTS FOR (a:Album) ON EACH [a.album]
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

#### Import Genres

In [8]:
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///genres_202206062119.csv' AS row FIELDTERMINATOR ','
WITH toInteger(row.genreid) as genreid, row.genre as genre SKIP 0 LIMIT 150000
CREATE (n:Genre {id: genreid, genre: genre})
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

#### Import Artists

In [9]:
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///artists_202206062119.csv' AS row FIELDTERMINATOR ','
WITH toInteger(row.artistid) as artistid, row.artist as artist SKIP 0 LIMIT 150000
CREATE (n:Artist {id: artistid, artist: artist})
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

Create fulltext index enabling fuzzy search:

In [10]:
query = """
CREATE FULLTEXT INDEX artists IF NOT EXISTS FOR (a:Artist) ON EACH [a.artist]
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

#### Import Songs

Songs with a trailing backslash will couse problems because the closing quote will be escaped. To mitigate this, I added an extra whitespace character to those song titles.

``` sql
SELECT * 
from cddb.songs s 
where song like '%\\' 
  and song not like '%\\\\'
```

Statement to update the rows:

``` sql
UPDATE cddb.songs 
SET song = song || ' ' 
WHERE song LIKE '%\\' 
  AND song NOT LIKE '%\\\\'
```

In [11]:
query = """
USING PERIODIC COMMIT 5000
LOAD CSV WITH HEADERS FROM 'file:///songs_202206072212.csv' AS row FIELDTERMINATOR ','
WITH toInteger(row.songid) as songid, row.song as song
CREATE (n:Song {id: songid, song: song})
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

Create fulltext index enabling fuzzy search:

In [12]:
query = """
CREATE FULLTEXT INDEX songs IF NOT EXISTS FOR (a:Song) ON EACH [a.song]
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

#### Import CDs 

In [13]:
query = """
USING PERIODIC COMMIT 5000
LOAD CSV WITH HEADERS FROM 'file:///cds_202206062119.csv' AS row FIELDTERMINATOR ','
WITH toInteger(row.cdid) as cdid, 
     toInteger(row.artist2albumid) as artist2albumid,
               toInteger(row.ayear)  as ayear,
     toInteger(row.genreid) as genreid,
               row.discid  as discid
CREATE (n:CD {id: cdid, artist2albumid: artist2albumid, ayear: ayear, genreid: genreid, discid: discid})
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

#### Connect CDs to Songs

In [14]:
query = """
CREATE INDEX IF NOT EXISTS FOR (n:CD) ON (n.id)
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

query = """
CREATE INDEX IF NOT EXISTS FOR (n:Song) ON (n.id)
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

In [ ]:
query = """
USING PERIODIC COMMIT 5000
LOAD CSV WITH HEADERS FROM 'file:///cdtracks_202206062119.csv' AS row FIELDTERMINATOR ','
WITH toInteger(row.songid) as songid, 
     toInteger(row.cdid) as cdid,
     toInteger(row.track) as track
MATCH (c:CD {id: cdid})
MATCH (s:Song {id: songid})
CREATE (c)-[r:CONTAINS {track: track}]->(s)
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

In [ ]:
query = """
MATCH (c:CD)-[r:CONTAINS]->(s:Song)
RETURN count(*)
"""
        
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data

**Other Direction:**

In [3]:
query = """
USING PERIODIC COMMIT 5000
LOAD CSV WITH HEADERS FROM 'file:///cdtracks_202206062119.csv' AS row FIELDTERMINATOR ','
WITH toInteger(row.songid) as songid, 
     toInteger(row.cdid) as cdid,
     toInteger(row.track) as track
MATCH (c:CD {id: cdid})
MATCH (s:Song {id: songid})
CREATE (c)<-[r:APPEARED_ON {track: track}]-(s)
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

In [4]:
query = """
MATCH (c:CD)<-[r:APPEARED_ON]-(s:Song)
RETURN count(*)
"""
        
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data

,count(*)
0,1163691


#### Connect CDs to Genre

In [5]:
query = """
CREATE INDEX IF NOT EXISTS FOR (n:Genre) ON (n.id)
"""

with driver.session(database=database_name) as session:
    result = session.run(query)


In [6]:
query = """
MATCH (c:CD)
MATCH (g:Genre {id: c.genreid})
CREATE (c)-[r:BELONGS_TO]->(g)
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

In [7]:
query = """
MATCH (c:CD)-[r:BELONGS_TO]->(s:Genre)
RETURN count(*)
"""
        
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data

,count(*)
0,121424


**Other Direction:**

In [8]:
query = """
MATCH (c:CD)
MATCH (g:Genre {id: c.genreid})
CREATE (c)<-[r:APPEARED_ON]-(g)
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

In [9]:
query = """
MATCH (c:CD)<-[r:APPEARED_ON]-(s:Genre)
RETURN count(*)
"""
        
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data

,count(*)
0,121424


#### Connect Artist, Album and CD

**Artist and Album:**

In [10]:
query = """
CREATE INDEX IF NOT EXISTS FOR (n:Album) ON (n.id)
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

In [11]:
query = """
CREATE INDEX IF NOT EXISTS FOR (n:Artist) ON (n.id)
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

In [12]:
query = """
USING PERIODIC COMMIT 5000
LOAD CSV WITH HEADERS FROM 'file:///artist2album_202206062119.csv' AS row FIELDTERMINATOR ','
WITH toInteger(row.artist2albumid) as artist2albumid, 
     toInteger(row.artistid) as artistid,
     toInteger(row.albumid) as albumid
MATCH (a:Artist {id: artistid})
MATCH (t:Album {id: albumid})
CREATE (a)-[r:CONTRIBUTED_TO]->(t)
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

**Other Direction:**

In [13]:
query = """
USING PERIODIC COMMIT 5000
LOAD CSV WITH HEADERS FROM 'file:///artist2album_202206062119.csv' AS row FIELDTERMINATOR ','
WITH toInteger(row.artist2albumid) as artist2albumid, 
     toInteger(row.artistid) as artistid,
     toInteger(row.albumid) as albumid
MATCH (a:Artist {id: artistid})
MATCH (t:Album {id: albumid})
CREATE (a)<-[r:CREATED_BY]-(t)
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

***

**CD and Album:**

In [14]:
query = """
CREATE INDEX IF NOT EXISTS FOR (n:CD) ON (n.artist2albumid)
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

In [15]:
query = """
USING PERIODIC COMMIT 5000
LOAD CSV WITH HEADERS FROM 'file:///artist2album_202206062119.csv' AS row FIELDTERMINATOR ','
WITH toInteger(row.artist2albumid) as artist2albumid, 
     toInteger(row.artistid) as artistid,
     toInteger(row.albumid) as albumid
MATCH (a:CD {artist2albumid: artist2albumid})
MATCH (t:Album {id: albumid})
CREATE (a)-[r:CONTAINS]->(t)
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

**Other Direction:**

In [16]:
query = """
USING PERIODIC COMMIT 5000
LOAD CSV WITH HEADERS FROM 'file:///artist2album_202206062119.csv' AS row FIELDTERMINATOR ','
WITH toInteger(row.artist2albumid) as artist2albumid, 
     toInteger(row.artistid) as artistid,
     toInteger(row.albumid) as albumid
MATCH (a:CD {artist2albumid: artist2albumid})
MATCH (t:Album {id: albumid})
CREATE (a)<-[r:APPEARED_ON]-(t)
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

***

**CD and Artist:**

In [17]:
query = """
USING PERIODIC COMMIT 5000
LOAD CSV WITH HEADERS FROM 'file:///artist2album_202206062119.csv' AS row FIELDTERMINATOR ','
WITH toInteger(row.artist2albumid) as artist2albumid, 
     toInteger(row.artistid) as artistid,
     toInteger(row.albumid) as albumid
MATCH (a:CD {artist2albumid: artist2albumid})
MATCH (t:Artist {id: artistid})
CREATE (a)-[r:CONTAINS]->(t)
"""

with driver.session(database=database_name) as session:
    result = session.run(query)

**Other Direction:**

In [18]:
query = """
USING PERIODIC COMMIT 5000
LOAD CSV WITH HEADERS FROM 'file:///artist2album_202206062119.csv' AS row FIELDTERMINATOR ','
WITH toInteger(row.artist2albumid) as artist2albumid, 
     toInteger(row.artistid) as artistid,
     toInteger(row.albumid) as albumid
MATCH (a:CD {artist2albumid: artist2albumid})
MATCH (t:Artist {id: artistid})
CREATE (a)<-[r:APPEARED_ON]-(t)
"""

with driver.session(database=database_name) as session:
    result = session.run(query)